In [1]:
pip install torchnet torch numpy pandas scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 19.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.2/301.2 kB 16.9 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
from torch import Tensor
from torch.utils import data
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import numpy as np
import datetime as dt
import pickle as pkl

import copy
import os
import json
from sklearn.model_selection import KFold
from dataset import PixelSetData_preloaded
# class PixelSetData(data.Dataset):
#     def __init__(self, folder, labels, npixel, sub_classes=None, norm=None,
#                  extra_feature=None, jitter=(0.01, 0.05), return_id=False):
#         """

#         Args:
#             folder (str): path to the main folder of the dataset, formatted as indicated in the readme
#             labels (str): name of the nomenclature to use in the labels.json file
#             npixel (int): Number of sampled pixels in each parcel
#             sub_classes (list): If provided, only the samples from the given list of classes are considered.
#             (Can be used to remove classes with too few samples)
#             norm (tuple): (mean,std) tuple to use for normalization
#             extra_feature (str): name of the additional static feature file to use
#             jitter (tuple): if provided (sigma, clip) values for the addition random gaussian noise
#             return_id (bool): if True, the id of the yielded item is also returned (useful for inference)
#         """
#         super(PixelSetData, self).__init__()

#         self.folder = folder
#         self.data_folder = os.path.join(folder, 'DATA')
#         self.meta_folder = os.path.join(folder, 'META')
#         self.labels = labels
#         self.npixel = npixel
#         self.norm = norm

#         self.extra_feature = extra_feature
#         self.jitter = jitter  # (sigma , clip )
#         self.return_id = return_id

#         l = [f for f in os.listdir(self.data_folder) if f.endswith('.npy')]
#         self.pid = [int(f.split('.')[0]) for f in l]
#         self.pid = list(np.sort(self.pid))

#         self.pid = list(map(str, self.pid))
#         self.len = len(self.pid)

#         # Get Labels
#         if sub_classes is not None:
#           sub_indices = []
#           num_classes = len(sub_classes)
#           convert = dict((c, i) for i, c in enumerate(sub_classes))
#           with open(os.path.join(folder, 'META', 'labels.json'), 'r') as file:
#             d = json.loads(file.read())
#             self.target = []
#             for i, p in enumerate(self.pid):
#               t = d[labels][p]
#               self.target.append(t)
#               if sub_classes is not None:
#                 if t in sub_classes:
#                   sub_indices.append(i)
#                   self.target[-1] = convert[self.target[-1]]
#         else:
#           self.target = []

#         if sub_classes is not None:
#             self.pid = list(np.array(self.pid)[sub_indices])
#             self.target = list(np.array(self.target)[sub_indices])
#             self.len = len(sub_indices)

#         with open(os.path.join(folder, 'META', 'dates.json'), 'r') as file:
#             d = json.loads(file.read())
#         self.dates = [d[str(i)] for i in range(len(d))]
#         self.date_positions = date_positions(self.dates)

#         if self.extra_feature is not None:
#             with open(os.path.join(self.meta_folder, '{}.json'.format(extra_feature)), 'r') as file:
#                 self.extra = json.loads(file.read())

#             if isinstance(self.extra[list(self.extra.keys())[0]], int):
#                 for k in self.extra.keys():
#                     self.extra[k] = [self.extra[k]]
#             df = pd.DataFrame(self.extra).transpose()
#             self.extra_m, self.extra_s = np.array(df.mean(axis=0)), np.array(df.std(axis=0))

#     def __len__(self):
#         return self.len

#     def __getitem__(self, item):
#         """
#         Returns a Pixel-Set sequence tensor with its pixel mask and optional additional features.
#         For each item npixel pixels are randomly dranw from the available pixels.
#         If the total number of pixel is too small one arbitrary pixel is repeated. The pixel mask keeps track of true
#         and repeated pixels.
#         Returns:
#               (Pixel-Set, Pixel-Mask) or ((Pixel-Set, Pixel-Mask), Extra-features) with:
#                 Pixel-Set: Sequence_length x Channels x npixel
#                 Pixel-Mask : Sequence_length x npixel
#                 Extra-features : Sequence_length x Number of additional features

#         """
#         x0 = np.load(os.path.join(self.folder, 'DATA', '{}.npy'.format(self.pid[item])))
#         if len(self.target) > 0:
#           y = self.target[item]
#         else:
#           y = 1

#         if x0.shape[-1] > self.npixel:
#             idx = np.random.choice(list(range(x0.shape[-1])), size=self.npixel, replace=False)
#             x = x0[:, :, idx]
#             mask = np.ones(self.npixel)

#         elif x0.shape[-1] < self.npixel:

#             if x0.shape[-1] == 0:
#                 x = np.zeros((*x0.shape[:2], self.npixel))
#                 mask = np.zeros(self.npixel)
#                 mask[0] = 1
#             else:
#                 x = np.zeros((*x0.shape[:2], self.npixel))
#                 x[:, :, :x0.shape[-1]] = x0
#                 x[:, :, x0.shape[-1]:] = np.stack([x[:, :, 0] for _ in range(x0.shape[-1], x.shape[-1])], axis=-1)
#                 mask = np.array(
#                     [1 for _ in range(x0.shape[-1])] + [0 for _ in range(x0.shape[-1], self.npixel)])
#         else:
#             x = x0
#             mask = np.ones(self.npixel)

#         if self.norm is not None:
#             m, s = self.norm
#             m = np.array(m)
#             s = np.array(s)

#             if len(m.shape) == 0:
#                 x = (x - m) / s
#             elif len(m.shape) == 1:  # Normalise channel-wise
#                 x = (x.swapaxes(1, 2) - m) / s
#                 x = x.swapaxes(1, 2)  # Normalise channel-wise for each date
#             elif len(m.shape) == 2:
#                 x = np.rollaxis(x, 2)  # TxCxS -> SxTxC
#                 x = (x - m) / s
#                 x = np.swapaxes((np.rollaxis(x, 1)), 1, 2)
#         x = x.astype('float')

#         if self.jitter is not None:
#             sigma, clip = self.jitter
#             x = x + np.clip(sigma * np.random.randn(*x.shape), -1 * clip, clip)

#         mask = np.stack([mask for _ in range(x.shape[0])], axis=0)  # Add temporal dimension to mask
#         data = (Tensor(x), Tensor(mask))

#         if self.extra_feature is not None:
#             ef = (self.extra[str(self.pid[item])] - self.extra_m) / self.extra_s
#             ef = torch.from_numpy(ef).float()

#             ef = torch.stack([ef for _ in range(data[0].shape[0])], dim=0)
#             data = (data, ef)

#         if self.return_id:
#             return data, torch.from_numpy(np.array(y, dtype=int)), self.pid[item]
#         else:
#             return data, torch.from_numpy(np.array(y, dtype=int))


# class PixelSetData_preloaded(PixelSetData):
#     """ Wrapper class to load all the dataset to RAM at initialization (when the hardware permits it).
#     """

#     def __init__(self, folder, labels, npixel, sub_classes=None, norm=None,
#                  extra_feature=None, jitter=(0.01, 0.05), return_id=False):
#         super(PixelSetData_preloaded, self).__init__(folder, labels, npixel, sub_classes=sub_classes, norm=norm,
#                  extra_feature=extra_feature, jitter=jitter, return_id=return_id)
#         self.samples = []
#         print('Loading samples to memory . . .')
#         for item in range(len(self)):
#             self.samples.append(super(PixelSetData_preloaded, self).__getitem__(item))
#         print('Done !')

#     def __getitem__(self, item):
#         return self.samples[item]


# def parse(date):
#     d = str(date)
#     return int(d[:4]), int(d[4:6]), int(d[6:])


# def interval_days(date1, date2):
#     return abs((dt.datetime(*parse(date1)) - dt.datetime(*parse(date2))).days)


# def date_positions(dates):
#     pos = []
#     for d in dates:
#         pos.append(interval_days(d, dates[0]))
#     return pos

In [2]:
subset = [1, 3, 4, 5, 6, 8, 9, 12, 13, 14, 16, 18, 19, 23, 28, 31, 33, 34, 36, 39]
# mean_std = pkl.load(open('./PixelSet-TOY/S2-2017-T31TFM-meanstd.pkl', 'rb'))
# dt1 = PixelSetData_preloaded('./PixelSet-TOY/', labels='label_44class', npixel=64,
#                                     sub_classes=subset,
#                                     norm=mean_std,
#                                     extra_feature=None)
mean_std = pkl.load(open('./pixelset_train/S2-2017-T31TFM-meanstd.pkl', 'rb'))
dt1 = PixelSetData_preloaded('./pixelset_train/', labels='label_19class', npixel=64,
                                    sub_classes=None,
                                    norm=mean_std,
                                    extra_feature=None)

Loading samples to memory . . .
Done !


In [3]:
# subset = [1, 3, 4, 5, 6, 8, 9, 12, 13, 14, 16, 18, 19, 23, 28, 31, 33, 34, 36, 39]
# mean_std = pkl.load(open('./PixelSet-TOY/S2-2017-T31TFM-meanstd.pkl', 'rb'))
# dttest = PixelSetData_preloaded('./test/', labels='label_19class', npixel=64,
#                                     sub_classes=None,
#                                     norm=mean_std,
#                                     extra_feature=None)

Loading samples to memory . . .
Done !


In [3]:
def get_loaders(dt, kfold):
    indices = list(range(len(dt)))
    np.random.shuffle(indices)

    kf = KFold(n_splits=kfold, shuffle=False)
    indices_seq = list(kf.split(list(range(len(dt)))))
    ntest = len(indices_seq[0][1])

    loader_seq = []
    for trainval, test_indices in indices_seq:
        trainval = [indices[i] for i in trainval]
        test_indices = [indices[i] for i in test_indices]

        validation_indices = trainval[-ntest:]
        train_indices = trainval[:-ntest]

        train_sampler = data.sampler.SubsetRandomSampler(train_indices)
        validation_sampler = data.sampler.SubsetRandomSampler(validation_indices)
        test_sampler = data.sampler.SubsetRandomSampler(test_indices)

        train_loader = data.DataLoader(dt, batch_size=128,
                                       sampler=train_sampler,
                                       num_workers=8)
        validation_loader = data.DataLoader(dt, batch_size=128,
                                            sampler=validation_sampler,
                                            num_workers=8)
        test_loader = data.DataLoader(dt, batch_size=128,
                                      sampler=test_sampler,
                                      num_workers=8)

        loader_seq.append((train_loader, validation_loader, test_loader))
    return loader_seq

In [4]:
class PixelSetEncoder(nn.Module):
    def __init__(self, input_dim, mlp1=[10, 32, 64], pooling='mean_std', mlp2=[64, 128], with_extra=True,
                 extra_size=4):
        """
        Pixel-set encoder.
        Args:
            input_dim (int): Number of channels of the input tensors
            mlp1 (list):  Dimensions of the successive feature spaces of MLP1
            pooling (str): Pixel-embedding pooling strategy, can be chosen in ('mean','std','max,'min')
                or any underscore-separated combination thereof.
            mlp2 (list): Dimensions of the successive feature spaces of MLP2
            with_extra (bool): Whether additional pre-computed features are passed between the two MLPs
            extra_size (int, optional): Number of channels of the additional features, if any.
        """

        super(PixelSetEncoder, self).__init__()

        self.input_dim = input_dim
        self.mlp1_dim = copy.deepcopy(mlp1)
        self.mlp2_dim = copy.deepcopy(mlp2)
        self.pooling = pooling

        self.with_extra = with_extra
        self.extra_size = extra_size

        self.name = 'PSE-{}-{}-{}'.format('|'.join(list(map(str, self.mlp1_dim))), pooling,
                                          '|'.join(list(map(str, self.mlp2_dim))))

        self.output_dim = input_dim * len(pooling.split('_')) if len(self.mlp2_dim) == 0 else self.mlp2_dim[-1]

        inter_dim = self.mlp1_dim[-1] * len(pooling.split('_'))


        if self.with_extra:
            self.name += 'Extra'
            inter_dim += self.extra_size
        print(inter_dim)
        print(mlp2[0])
        assert (input_dim == mlp1[0])
        assert (inter_dim == mlp2[0])
        # Feature extraction
        layers = []
        for i in range(len(self.mlp1_dim) - 1):
            layers.append(linlayer(self.mlp1_dim[i], self.mlp1_dim[i + 1]))
        self.mlp1 = nn.Sequential(*layers)

        # MLP after pooling
        layers = []
        for i in range(len(self.mlp2_dim) - 1):
            layers.append(nn.Linear(self.mlp2_dim[i], self.mlp2_dim[i + 1]))
            layers.append(nn.BatchNorm1d(self.mlp2_dim[i + 1]))
            if i < len(self.mlp2_dim) - 2:
                layers.append(nn.ReLU())
        self.mlp2 = nn.Sequential(*layers)

    def forward(self, input):
        """
        The input of the PSE is a tuple of tensors as yielded by the PixelSetData class:
          (Pixel-Set, Pixel-Mask) or ((Pixel-Set, Pixel-Mask), Extra-features)
        Pixel-Set : Batch_size x (Sequence length) x Channel x Number of pixels
        Pixel-Mask : Batch_size x (Sequence length) x Number of pixels
        Extra-features : Batch_size x (Sequence length) x Number of features

        If the input tensors have a temporal dimension, it will be combined with the batch dimension so that the
        complete sequences are processed at once. Then the temporal dimension is separated back to produce a tensor of
        shape Batch_size x Sequence length x Embedding dimension
        """
        a, b = input
        if len(a) == 2:
            out, mask = a
            extra = b
            if len(extra) == 2:
                extra, bm = extra
        else:
            out, mask = a, b

        if len(out.shape) == 4:
            # Combine batch and temporal dimensions in case of sequential input
            reshape_needed = True
            batch, temp = out.shape[:2]

            out = out.view(batch * temp, *out.shape[2:])
            mask = mask.view(batch * temp, -1)
            if self.with_extra:
                extra = extra.view(batch * temp, -1)
        else:
            reshape_needed = False

        out = self.mlp1(out)
        out = torch.cat([pooling_methods[n](out, mask) for n in self.pooling.split('_')], dim=1)

        if self.with_extra:
            out = torch.cat([out, extra], dim=1)
        out = self.mlp2(out)

        if reshape_needed:
            out = out.view(batch, temp, -1)
        return out

class linlayer(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(linlayer, self).__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim

        self.lin = nn.Linear(in_dim, out_dim)
        self.bn = nn.BatchNorm1d(out_dim)

    def forward(self, input):
        out = input.permute((0, 2, 1))  # to channel last
        out = self.lin(out)

        out = out.permute((0, 2, 1))  # to channel first
        out = self.bn(out)
        out = F.relu(out)

        return out

def masked_mean(x, mask):
    out = x.permute((1, 0, 2))
    out = out * mask
    out = out.sum(dim=-1) / mask.sum(dim=-1)
    out = out.permute((1, 0))
    return out

def masked_std(x, mask):
    m = masked_mean(x, mask)

    out = x.permute((2, 0, 1))
    out = out - m
    out = out.permute((2, 1, 0))

    out = out * mask
    d = mask.sum(dim=-1)
    d[d == 1] = 2

    out = (out ** 2).sum(dim=-1) / (d - 1)
    out = torch.sqrt(out + 10e-32) # To ensure differentiability
    out = out.permute(1, 0)
    return out

def maximum(x, mask):
    return x.max(dim=-1)[0].squeeze()

def minimum(x, mask):
    return x.min(dim=-1)[0].squeeze()

pooling_methods = {
    'mean': masked_mean,
    'std': masked_std,
    'max': maximum,
    'min': minimum
}

In [5]:
class LTAE(nn.Module):
    def __init__(self, in_channels=128, n_head=16, d_k=8, n_neurons=[256,128], dropout=0.2, d_model=256,
                 T=1000, len_max_seq=24, positions=None, return_att=False):
        """
        Sequence-to-embedding encoder.
        Args:
            in_channels (int): Number of channels of the input embeddings
            n_head (int): Number of attention heads
            d_k (int): Dimension of the key and query vectors
            n_neurons (list): Defines the dimensions of the successive feature spaces of the MLP that processes
                the concatenated outputs of the attention heads
            dropout (float): dropout
            T (int): Period to use for the positional encoding
            len_max_seq (int, optional): Maximum sequence length, used to pre-compute the positional encoding table
            positions (list, optional): List of temporal positions to use instead of position in the sequence
            d_model (int, optional): If specified, the input tensors will first processed by a fully connected layer
                to project them into a feature space of dimension d_model
            return_att (bool): If true, the module returns the attention masks along with the embeddings (default False)

        """

        super(LTAE, self).__init__()
        self.in_channels = in_channels
        self.positions = positions
        self.n_neurons = copy.deepcopy(n_neurons)
        self.return_att = return_att


        if positions is None:
            positions = len_max_seq + 1

        if d_model is not None:
            self.d_model = d_model
            self.inconv = nn.Sequential(nn.Conv1d(in_channels, d_model, 1),
                                        nn.LayerNorm((d_model, len_max_seq)))
        else:
            self.d_model = in_channels
            self.inconv = None

        sin_tab = get_sinusoid_encoding_table(positions, self.d_model // n_head, T=T)
        self.position_enc = nn.Embedding.from_pretrained(torch.cat([sin_tab for _ in range(n_head)], dim=1),
                                                         freeze=True)

        self.inlayernorm = nn.LayerNorm(self.in_channels)

        self.outlayernorm = nn.LayerNorm(n_neurons[-1])

        self.attention_heads = MultiHeadAttention(
            n_head=n_head, d_k=d_k, d_in=self.d_model)

        assert (self.n_neurons[0] == self.d_model)

        activation = nn.ReLU()

        layers = []
        for i in range(len(self.n_neurons) - 1):
            layers.extend([nn.Linear(self.n_neurons[i], self.n_neurons[i + 1]),
                           nn.BatchNorm1d(self.n_neurons[i + 1]),
                           activation])

        self.mlp = nn.Sequential(*layers)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):

        sz_b, seq_len, d = x.shape

        x = self.inlayernorm(x)

        if self.inconv is not None:
            x = self.inconv(x.permute(0, 2, 1)).permute(0, 2, 1)

        if self.positions is None:
            src_pos = torch.arange(1, seq_len + 1, dtype=torch.long).expand(sz_b, seq_len).to(x.device)
        else:
            src_pos = torch.arange(0, seq_len, dtype=torch.long).expand(sz_b, seq_len).to(x.device)
        enc_output = x + self.position_enc(src_pos)

        enc_output, attn = self.attention_heads(enc_output, enc_output, enc_output)

        enc_output = enc_output.permute(1, 0, 2).contiguous().view(sz_b, -1)  # Concatenate heads

        enc_output = self.outlayernorm(self.dropout(self.mlp(enc_output)))

        if self.return_att:
            return enc_output, attn
        else:
            return enc_output


class MultiHeadAttention(nn.Module):
    ''' Multi-Head Attention module '''

    def __init__(self, n_head, d_k, d_in):
        super().__init__()
        self.n_head = n_head
        self.d_k = d_k
        self.d_in = d_in

        self.Q = nn.Parameter(torch.zeros((n_head, d_k))).requires_grad_(True)
        nn.init.normal_(self.Q, mean=0, std=np.sqrt(2.0 / (d_k)))

        self.fc1_k = nn.Linear(d_in, n_head * d_k)
        nn.init.normal_(gself.fc1_k.weight, mean=0, std=np.sqrt(2.0 / (d_k)))

        self.attention = ScaledDotProductAttention(temperature=np.power(d_k, 0.5))

    def forward(self, q, k, v):
        d_k, d_in, n_head = self.d_k, self.d_in, self.n_head
        sz_b, seq_len, _ = q.size()

        q = torch.stack([self.Q for _ in range(sz_b)], dim=1).view(-1, d_k)  # (n*b) x d_k

        k = self.fc1_k(v).view(sz_b, seq_len, n_head, d_k)
        k = k.permute(2, 0, 1, 3).contiguous().view(-1, seq_len, d_k)  # (n*b) x lk x dk

        v = torch.stack(v.split(v.shape[-1] // n_head, dim=-1)).view(n_head * sz_b, seq_len, -1)
        output, attn = self.attention(q, k, v)
        attn = attn.view(n_head, sz_b, 1, seq_len)
        attn = attn.squeeze(dim=2)

        output = output.view(n_head, sz_b, 1, d_in // n_head)
        output = output.squeeze(dim=2)

        return output, attn


class ScaledDotProductAttention(nn.Module):
    ''' Scaled Dot-Product Attention '''

    def __init__(self, temperature, attn_dropout=0.1):
        super().__init__()
        self.temperature = temperature
        self.dropout = nn.Dropout(attn_dropout)
        self.softmax = nn.Softmax(dim=2)

    def forward(self, q, k, v):
        attn = torch.matmul(q.unsqueeze(1), k.transpose(1, 2))
        attn = attn / self.temperature

        attn = self.softmax(attn)
        attn = self.dropout(attn)
        output = torch.matmul(attn, v)

        return output, attn


def get_sinusoid_encoding_table(positions, d_hid, T=1000):
    ''' Sinusoid position encoding table
    positions: int or list of integer, if int range(positions)'''

    if isinstance(positions, int):
        positions = list(range(positions))

    def cal_angle(position, hid_idx):
        return position / np.power(T, 2 * (hid_idx // 2) / d_hid)

    def get_posi_angle_vec(position):
        return [cal_angle(position, hid_j) for hid_j in range(d_hid)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in positions])

    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # dim 2i
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # dim 2i+1

    if torch.cuda.is_available():
        return torch.FloatTensor(sinusoid_table).cuda()
    else:
        return torch.FloatTensor(sinusoid_table)


def get_sinusoid_encoding_table_var(positions, d_hid, clip=4, offset=3, T=1000):
    ''' Sinusoid position encoding table
    positions: int or list of integer, if int range(positions)'''

    if isinstance(positions, int):
        positions = list(range(positions))

    x = np.array(positions)

    def cal_angle(position, hid_idx):
        return position / np.power(T, 2 * (hid_idx + offset // 2) / d_hid)

    def get_posi_angle_vec(position):
        return [cal_angle(position, hid_j) for hid_j in range(d_hid)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in positions])

    sinusoid_table = np.sin(sinusoid_table)  # dim 2i
    sinusoid_table[:, clip:] = torch.zeros(sinusoid_table[:, clip:].shape)

    if torch.cuda.is_available():
        return torch.FloatTensor(sinusoid_table).cuda()
    else:
        return torch.FloatTensor(sinusoid_table)

In [6]:
def get_decoder(n_neurons):
    """Returns an MLP with the layer widths specified in n_neurons.
    Every linear layer but the last one is followed by BatchNorm + ReLu

    args:
        n_neurons (list): List of int that specifies the width and length of the MLP.
    """
    layers = []
    for i in range(len(n_neurons)-1):
        layers.append(nn.Linear(n_neurons[i], n_neurons[i+1]))
        if i < (len(n_neurons) - 2):
            layers.extend([
                nn.BatchNorm1d(n_neurons[i + 1]),
                nn.ReLU()
            ])
    m = nn.Sequential(*layers)
    return m

In [7]:
def get_ntrainparams(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

class PseLTae(nn.Module):
    """
    Pixel-Set encoder + Lightweight Temporal Attention Encoder sequence classifier
    """

    def __init__(self, input_dim=10, mlp1=[10, 32, 64], pooling='mean_std', mlp2=[132, 128], with_extra=True,
                 extra_size=4,
                 n_head=16, d_k=8, d_model=256, mlp3=[256, 128], dropout=0.2, T=1000, len_max_seq=24, positions=None,
                 mlp4=[128, 64, 32, 20], return_att=False):
        super(PseLTae, self).__init__()
        self.spatial_encoder = PixelSetEncoder(input_dim, mlp1=mlp1, pooling=pooling, mlp2=mlp2, with_extra=with_extra,
                                               extra_size=extra_size)
        self.temporal_encoder = LTAE(in_channels=mlp2[-1], n_head=n_head, d_k=d_k,
                                           d_model=d_model, n_neurons=mlp3, dropout=dropout,
                                           T=T, len_max_seq=len_max_seq, positions=positions, return_att=return_att
                                           )
        self.decoder = get_decoder(mlp4)
        self.return_att = return_att

    def forward(self, input):
        """
         Args:
            input(tuple): (Pixel-Set, Pixel-Mask) or ((Pixel-Set, Pixel-Mask), Extra-features)
            Pixel-Set : Batch_size x Sequence length x Channel x Number of pixels
            Pixel-Mask : Batch_size x Sequence length x Number of pixels
            Extra-features : Batch_size x Sequence length x Number of features
        """
        out = self.spatial_encoder(input)
        if self.return_att:
            out, att = self.temporal_encoder(out)
            out = self.decoder(out)
            return out, att
        else:
            out = self.temporal_encoder(out)
            out = self.decoder(out)
            return out

    def param_ratio(self):
        total = get_ntrainparams(self)
        s = get_ntrainparams(self.spatial_encoder)
        t = get_ntrainparams(self.temporal_encoder)
        c = get_ntrainparams(self.decoder)

        print('TOTAL TRAINABLE PARAMETERS : {}'.format(total))
        print('RATIOS: Spatial {:5.1f}% , Temporal {:5.1f}% , Classifier {:5.1f}%'.format(s / total * 100,
                                                                                          t / total * 100,
                                                                                          c / total * 100))

        return total

In [8]:
device = torch.device('cpu')
loaders = get_loaders(dt1, 5)

In [9]:
for fold, (train_loader, val_loader, test_loader) in enumerate(loaders):
  print('Starting Fold {}'.format(fold + 1))
  print('Train {}, Val {}, Test {}'.format(len(train_loader), len(val_loader), len(test_loader)))

  model_config = dict(input_dim=10, mlp1=[10,32,64], pooling='mean_std',
                                  mlp2=[128, 132], n_head=16, d_k=8, mlp3=[256,128],
                                  dropout=0.2, T=1000, len_max_seq=24,
                                  positions=None,
                                  mlp4=[128, 64, 32, 20], d_model=256)
  model_config.update(with_extra=False, extra_size=None)
  model = PseLTae(**model_config)

Starting Fold 1
Train 901, Val 301, Test 301
128
128
Starting Fold 2
Train 901, Val 301, Test 301
128
128
Starting Fold 3
Train 901, Val 301, Test 301
128
128
Starting Fold 4
Train 901, Val 301, Test 301
128
128
Starting Fold 5
Train 901, Val 301, Test 301
128
128


In [10]:
import torch.nn.init as init
def weight_init(m):
    '''
    Initializes a model's parameters.
    Credits to: https://gist.github.com/jeasinema

    Usage:
        model = Model()
        model.apply(weight_init)
    '''
    if isinstance(m, nn.Conv1d):
        init.normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.Conv2d):
        init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.Conv3d):
        init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.ConvTranspose1d):
        init.normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.ConvTranspose2d):
        init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.ConvTranspose3d):
        init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.BatchNorm1d):
        init.normal_(m.weight.data, mean=0, std=1)
        init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.BatchNorm2d):
        init.normal_(m.weight.data, mean=0, std=1)
        init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.BatchNorm3d):
        init.normal_(m.weight.data, mean=0, std=1)
        init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        init.xavier_normal_(m.weight.data)
        try:
            init.normal_(m.bias.data)
        except AttributeError:
            pass
    elif isinstance(m, nn.LSTM):
        for param in m.parameters():
            if len(param.shape) >= 2:
                init.orthogonal_(param.data)
            else:
                init.normal_(param.data)
    elif isinstance(m, nn.LSTMCell):
        for param in m.parameters():
            if len(param.shape) >= 2:
                init.orthogonal_(param.data)
            else:
                init.normal_(param.data)
    elif isinstance(m, nn.GRU):
        for param in m.parameters():
            if len(param.shape) >= 2:
                init.orthogonal_(param.data)
            else:
                init.normal_(param.data)
    elif isinstance(m, nn.GRUCell):
        for param in m.parameters():
            if len(param.shape) >= 2:
                init.orthogonal_(param.data)
            else:
                init.normal_(param.data)

In [11]:
from torch.autograd import Variable
class FocalLoss(nn.Module):
    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha, (float, int)): self.alpha = torch.Tensor([alpha, 1 - alpha])
        if isinstance(alpha, list): self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim() > 2:
            input = input.view(input.size(0), input.size(1), -1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1, 2)  # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1, input.size(2))  # N,H*W,C => N*H*W,C
        target = target.view(-1, 1)

        logpt = F.log_softmax(input, dim=1)
        logpt = logpt.gather(1, target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type() != input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0, target.data.view(-1))
            logpt = logpt * Variable(at)

        loss = -1 * (1 - pt) ** self.gamma * logpt
        if self.size_average:
            return loss.mean()
        else:
            return loss.sum()

In [12]:
def mIou(y_true, y_pred, n_classes):
    """
    Mean Intersect over Union metric.
    Computes the one versus all IoU for each class and returns the average.
    Classes that do not appear in the provided set are not counted in the average.
    Args:
        y_true (1D-array): True labels
        y_pred (1D-array): Predicted labels
        n_classes (int): Total number of classes
    Returns:
        mean Iou (float)
    """
    iou = 0
    n_observed = n_classes
    for i in range(n_classes):
        y_t = (np.array(y_true) == i).astype(int)
        y_p = (np.array(y_pred) == i).astype(int)

        inter = np.sum(y_t * y_p)
        union = np.sum((y_t + y_p > 0).astype(int))

        if union == 0:
            n_observed -= 1
        else:
            iou += inter / union

    return iou / n_observed



def confusion_matrix_analysis(mat):
    """
    This method computes all the performance metrics from the confusion matrix. In addition to overall accuracy, the
    precision, recall, f-score and IoU for each class is computed.
    The class-wise metrics are averaged to provide overall indicators in two ways (MICRO and MACRO average)
    Args:
        mat (array): confusion matrix

    Returns:
        per_class (dict) : per class metrics
        overall (dict): overall metrics

    """
    TP = 0
    FP = 0
    FN = 0

    per_class = {}

    for j in range(mat.shape[0]):
        d = {}
        tp = np.sum(mat[j, j])
        fp = np.sum(mat[:, j]) - tp
        fn = np.sum(mat[j, :]) - tp

        d['IoU'] = tp / (tp + fp + fn)
        d['Precision'] = tp / (tp + fp)
        d['Recall'] = tp / (tp + fn)
        d['F1-score'] = 2 * tp / (2 * tp + fp + fn)

        per_class[str(j)] = d

        TP += tp
        FP += fp
        FN += fn

    overall = {}
    overall['micro_IoU'] = TP / (TP + FP + FN)
    overall['micro_Precision'] = TP / (TP + FP)
    overall['micro_Recall'] = TP / (TP + FN)
    overall['micro_F1-score'] = 2 * TP / (2 * TP + FP + FN)

    macro = pd.DataFrame(per_class).transpose().mean()
    overall['MACRO_IoU'] = macro.loc['IoU']
    overall['MACRO_Precision'] = macro.loc['Precision']
    overall['MACRO_Recall'] = macro.loc['Recall']
    overall['MACRO_F1-score'] = macro.loc['F1-score']

    overall['Accuracy'] = np.sum(np.diag(mat)) / np.sum(mat)

    return per_class, overall

In [13]:
model = model.to(device)
model.apply(weight_init)
optimizer = torch.optim.Adam(model.parameters())
criterion = FocalLoss(1)

In [14]:
import torchnet as tnt
def recursive_todevice(x, device):
    if isinstance(x, torch.Tensor):
        return x.to(device)
    else:
        return [recursive_todevice(c, device) for c in x]

def evaluation(model, criterion, loader, device, mode='val'):
    y_true = []
    y_pred = []

    acc_meter = tnt.meter.ClassErrorMeter(accuracy=True)
    loss_meter = tnt.meter.AverageValueMeter()

    for (x, y) in loader:
        y_true.extend(list(map(int, y)))
        x = recursive_todevice(x, device)
        y = y.to(device)

        with torch.no_grad():
            prediction = model(x)
            loss = criterion(prediction, y)

        acc_meter.add(prediction, y)
        loss_meter.add(loss.item())

        y_p = prediction.argmax(dim=1).cpu().numpy()
        y_pred.extend(list(y_p))
        # print(y_pred)
        # print(y_true)

    metrics = {'{}_accuracy'.format(mode): acc_meter.value()[0],
               '{}_loss'.format(mode): loss_meter.value()[0],
               '{}_IoU'.format(mode): mIou(y_true, y_pred, 20)}

    if mode == 'val':
        return metrics
    # elif mode == 'test':
    #     return metrics, confusion_matrix(y_true, y_pred, labels=list(range(config['num_classes'])))

In [15]:
trainlog = {}
best_mIoU = 0
epochs = 100
for epoch in range(1, epochs + 1):
  print('EPOCH {}/{}'.format(epoch, epochs))
  model.train()
  print('Validation . . . ')
  model.eval()
  val_metrics = evaluation(model, criterion, val_loader, device=device, mode='val')
  print('Loss {:.4f},  Acc {:.2f},  IoU {:.4f}'.format(val_metrics['val_loss'], val_metrics['val_accuracy'],
                                                                 val_metrics['val_IoU']))
  # trainlog[epoch] = {**train_metrics, **val_metrics}
  # # checkpoint(fold + 1, trainlog, config)
  # if val_metrics['val_IoU'] >= best_mIoU:
  #   best_mIoU = val_metrics['val_IoU']
  #   torch.save({'epoch': epoch, 'state_dict': model.state_dict(),
  #             'optimizer': optimizer.state_dict()},
  #   os.path.join(config['res_dir'], 'Fold_{}'.format(fold + 1), 'model.pth.tar'))


EPOCH 1/100
Validation . . . 
Loss 3.2026,  Acc 0.03,  IoU 0.0000
EPOCH 2/100
Validation . . . 
Loss 3.2039,  Acc 0.03,  IoU 0.0000
EPOCH 3/100
Validation . . . 
Loss 3.2043,  Acc 0.03,  IoU 0.0000
EPOCH 4/100
Validation . . . 
Loss 3.2045,  Acc 0.03,  IoU 0.0000
EPOCH 5/100
Validation . . . 
Loss 3.2014,  Acc 0.03,  IoU 0.0000
EPOCH 6/100
Validation . . . 
Loss 3.2031,  Acc 0.03,  IoU 0.0000
EPOCH 7/100
Validation . . . 
Loss 3.2039,  Acc 0.03,  IoU 0.0000
EPOCH 8/100
Validation . . . 
Loss 3.2018,  Acc 0.03,  IoU 0.0000
EPOCH 9/100
Validation . . . 
Loss 3.2037,  Acc 0.03,  IoU 0.0000
EPOCH 10/100
Validation . . . 
Loss 3.2057,  Acc 0.03,  IoU 0.0000
EPOCH 11/100
Validation . . . 
Loss 3.2033,  Acc 0.03,  IoU 0.0000
EPOCH 12/100
Validation . . . 
Loss 3.2027,  Acc 0.03,  IoU 0.0000
EPOCH 13/100
Validation . . . 
Loss 3.2024,  Acc 0.03,  IoU 0.0000
EPOCH 14/100
Validation . . . 
Loss 3.2047,  Acc 0.03,  IoU 0.0000
EPOCH 15/100
Validation . . . 
Loss 3.2028,  Acc 0.03,  IoU 0.0000
EPOC

KeyboardInterrupt: 

In [17]:
# dttest[0]
!python --version

Python 3.10.12


In [ ]:
dl = data.DataLoader(dttest, batch_size=128, num_workers=8)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
from tqdm import tqdm
for (x, y) in tqdm(dl):
  ids = list(x)

  0%|          | 0/1 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|██████████| 1/1 [00:00<00:00,  3.29it/s]


In [ ]:
x[0].shape

torch.Size([1, 24, 10, 64])

In [ ]:
def recursive_todevice(x, device):
    if isinstance(x, torch.Tensor):
        return x.to(  device)
    else:
        return [recursive_todevice(c, device) for c in x]

In [ ]:
x = recursive_todevice(x, device='cpu')

In [ ]:
x[0].shape

torch.Size([1, 24, 10, 64])

In [ ]:
x[1]

tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]])

In [ ]:
pred = model(x)

In [ ]:
# x[0].shape
pred

tensor([[ 0.2368, -1.9760, -0.3465, -3.9236,  0.9039,  0.4160, -2.4715,  2.4870,
          1.1034, -0.9424,  0.9144,  1.6674, -0.9879, -0.8016,  3.0061, -1.5765,
          1.8263, -0.2311, -2.0076, -1.4873]], grad_fn=<AddmmBackward0>)

In [ ]:
# # prompt: read png file as flattened numpy array

# import numpy as np
# from PIL import Image

# # Open the PNG image
# image = Image.open('./new/2024-02-08-00:00_2024-02-08-23:59_Sentinel-2_L2A_B01_(Raw).png')

# # Convert the image to a numpy array
# image_array = np.asarray(image)

# # Flatten the numpy array
# flattened_array = image_array.flatten()

# # Print the flattened numpy array
# print(flattened_array)


11


In [ ]:
# prompt: read all files in a directory as numpy array

import os
import numpy as np
from PIL import Image
def read_all_npy_as_array(directory):
  """
  Reads all files in a directory as numpy arrays and returns a single array containing all the data.

  Args:
    directory (str): The directory containing the numpy files.

  Returns:
    np.ndarray: A single numpy array containing all the data from the files in the directory.
  """

  # Get a list of all the files in the directory
  files = os.listdir(directory)

  # Initialize an empty list to store the data from each file
  data = []

  # Loop through the files and read them as numpy arrays
  for file in files:
    file_path = os.path.join(directory, file)
    if len(data)>0:
      image = Image.open(file_path)
      datatem = np.asarray(image)
      datatem = datatem.flatten()
      data = np.add(data, datatem)
    else:
      image = Image.open(file_path)
      data = np.asarray(image)
      data = data.flatten()

  # Concatenate the data from all the files into a single array
  # data = np.concatenate(data, axis=0)

  return data


In [ ]:
t = np.load('./drive/MyDrive/PixelSet-TOY/DATA/101.npy')
t.shape

(24, 10, 304)

In [ ]:
import torch
from torch import Tensor
from torch.utils import data
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import numpy as np
import datetime as dt
import pickle as pkl

import copy
import os
import json
from sklearn.model_selection import KFold

class PixelSetDataTest(data.Dataset):
    def __init__(self, data, npixel):
        """

        Args:
            folder (str): path to the main folder of the dataset, formatted as indicated in the readme
            labels (str): name of the nomenclature to use in the labels.json file
            npixel (int): Number of sampled pixels in each parcel
            sub_classes (list): If provided, only the samples from the given list of classes are considered.
            (Can be used to remove classes with too few samples)
            norm (tuple): (mean,std) tuple to use for normalization
            extra_feature (str): name of the additional static feature file to use
            jitter (tuple): if provided (sigma, clip) values for the addition random gaussian noise
            return_id (bool): if True, the id of the yielded item is also returned (useful for inference)
        """
        super(PixelSetDataTest, self).__init__()

        # self.folder = folder
        # self.data_folder = os.path.join(folder, 'DATA')
        # self.meta_folder = os.path.join(folder, 'META')
        # self.labels = labels
        self.data = data
        self.npixel = npixel
        # self.norm = norm

        # self.extra_feature = extra_feature
        # self.jitter = jitter  # (sigma , clip )
        # self.return_id = return_id

        # l = [f for f in os.listdir(self.data_folder) if f.endswith('.npy')]
        # self.pid = [int(f.split('.')[0]) for f in l]
        # self.pid = list(np.sort(self.pid))

        # self.pid = list(map(str, self.pid))
        # self.len = len(self.pid)

        # with open(os.path.join(folder, 'META', 'dates.json'), 'r') as file:
        #     d = json.loads(file.read())
        # self.dates = [d[str(i)] for i in range(len(d))]
        # self.date_positions = date_positions(self.dates)

    def __len__(self):
        return 1

    def __getitem__(self, item):
        """
        Returns a Pixel-Set sequence tensor with its pixel mask and optional additional features.
        For each item npixel pixels are randomly dranw from the available pixels.
        If the total number of pixel is too small one arbitrary pixel is repeated. The pixel mask keeps track of true
        and repeated pixels.
        Returns:
              (Pixel-Set, Pixel-Mask) or ((Pixel-Set, Pixel-Mask), Extra-features) with:
                Pixel-Set: Sequence_length x Channels x npixel
                Pixel-Mask : Sequence_length x npixel
                Extra-features : Sequence_length x Number of additional features

        """
        # x0 = np.load(os.path.join(self.folder, 'DATA', '{}.npy'.format(self.pid[item])))
        x0 = self.data
        # if len(self.target) > 0:
        #   y = self.target[item]
        # else:
        #   y = []

        if x0.shape[-1] > self.npixel:
            idx = np.random.choice(list(range(x0.shape[-1])), size=self.npixel, replace=False)
            x = x0[:, :, idx]
            mask = np.ones(self.npixel)

        elif x0.shape[-1] < self.npixel:

            if x0.shape[-1] == 0:
                x = np.zeros((*x0.shape[:2], self.npixel))
                mask = np.zeros(self.npixel)
                mask[0] = 1
            else:
                x = np.zeros((*x0.shape[:2], self.npixel))
                x[:, :, :x0.shape[-1]] = x0
                x[:, :, x0.shape[-1]:] = np.stack([x[:, :, 0] for _ in range(x0.shape[-1], x.shape[-1])], axis=-1)
                mask = np.array(
                    [1 for _ in range(x0.shape[-1])] + [0 for _ in range(x0.shape[-1], self.npixel)])
        else:
            x = x0
            mask = np.ones(self.npixel)

        x = x.astype('float')


        mask = np.stack([mask for _ in range(x.shape[0])], axis=0)  # Add temporal dimension to mask
        data = (Tensor(x), Tensor(mask))
        return data, torch.from_numpy(np.array(y, dtype=int))


class PixelSetData_preloadedTest(PixelSetDataTest):
    """ Wrapper class to load all the dataset to RAM at initialization (when the hardware permits it).
    """

    def __init__(self, data, npixel):
        super(PixelSetData_preloadedTest, self).__init__(data, npixel)
        self.samples = []
        print('Loading samples to memory . . .')
        for item in range(len(self)):
            self.samples.append(super(PixelSetData_preloadedTest, self).__getitem__(item))
        print('Done !')

    def __getitem__(self, item):
        return self.samples[item]


def parse(date):
    d = str(date)
    return int(d[:4]), int(d[4:6]), int(d[6:])


def interval_days(date1, date2):
    return abs((dt.datetime(*parse(date1)) - dt.datetime(*parse(date2))).days)


def date_positions(dates):
    pos = []
    for d in dates:
        pos.append(interval_days(d, dates[0]))
    return pos

In [ ]:
data2 = read_all_npy_as_array("./new/");
data2 = np.expand_dims(data2, axis=0)
data2 = np.expand_dims(data2, axis=0)
data2.shape

FileNotFoundError: [Errno 2] No such file or directory: './new/'

In [ ]:
mean_std = pkl.load(open('./drive/MyDrive/PixelSet-TOY/S2-2017-T31TFM-meanstd.pkl', 'rb'))
dttest2 = PixelSetData_preloadedTest(data = data2, npixel=64)
dttest2

Loading samples to memory . . .
Done !


In [ ]:
dl2 = data.DataLoader(dttest2, batch_size=128, num_workers=8)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
from tqdm import tqdm
for (x2, y) in tqdm(dttest2):
  ids = list(x2)

100%|██████████| 1/1 [00:00<00:00, 5592.41it/s]


In [ ]:
x2[0].shape

torch.Size([1, 1, 64])

In [ ]:
model(x2)

NameError: name 'x2' is not defined

In [ ]:
data1 = np.load("./test/DATA/2.npy")

In [ ]:
data1.shape

(1, 10, 576)

In [41]:
mean_std = pkl.load(open('./drive/MyDrive/PixelSet-TOY/S2-2017-T31TFM-meanstd.pkl', 'rb'))
dttest1 = PixelSetData_preloadedTest(data = data1, npixel=64)

FileNotFoundError: [Errno 2] No such file or directory: './drive/MyDrive/PixelSet-TOY/S2-2017-T31TFM-meanstd.pkl'

In [ ]:
subset = [1, 3, 4, 5, 6, 8, 9, 12, 13, 14, 16, 18, 19, 23, 28, 31, 33, 34, 36, 39]
mean_std = pkl.load(open('./drive/MyDrive/PixelSet-TOY/S2-2017-T31TFM-meanstd.pkl', 'rb'))
dttest = PixelSetData_preloaded('./test/', labels='label_19class', npixel=64,
                                    sub_classes=None,
                                    norm=mean_std,
                                    extra_feature=None)

Loading samples to memory . . .
Done !


In [ ]:
dl = data.DataLoader(dttest, batch_size=128, num_workers=8)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
dl.batch_size

128

In [ ]:
from tqdm import tqdm
for (x, y) in tqdm(dl):
  ids = list(x)

100%|██████████| 1/1 [00:00<00:00,  3.38it/s]


In [ ]:
x = recursive_todevice(x, device='cpu')

In [ ]:
x[0].shape

torch.Size([1, 24, 10, 64])

In [ ]:
model(x)

tensor([[ 2.6035, -1.6439,  0.5844, -0.6367, -0.8194, -0.7185, -0.2194, -0.0597,
          0.0922, -0.4818, -2.1169,  2.0929, -1.0708,  1.7885,  1.0255,  1.4232,
          1.2237, -0.6139,  3.0601, -3.1487]], grad_fn=<AddmmBackward0>)

In [ ]:
torch.save(model.state_dict(), './model.pth')

In [ ]:
model_config = dict(input_dim=10, mlp1=[10,32,64], pooling='mean_std',
                                  mlp2=[128, 132], n_head=16, d_k=8, mlp3=[256,128],
                                  dropout=0.2, T=1000, len_max_seq=24,
                                  positions=None,
                                  mlp4=[128, 64, 32, 20], d_model=256)
model_config.update(with_extra=False, extra_size=None)
model2 = PseLTae(**model_config)

128
128


In [ ]:
torch.save(model, './model.pt')

In [ ]:
model2 = torch.load('./model.pt')

In [ ]:
model2(x)

tensor([[ 2.6035, -1.6439,  0.5844, -0.6367, -0.8194, -0.7185, -0.2194, -0.0597,
          0.0922, -0.4818, -2.1169,  2.0929, -1.0708,  1.7885,  1.0255,  1.4232,
          1.2237, -0.6139,  3.0601, -3.1487]], grad_fn=<AddmmBackward0>)

In [ ]:
!python --version

Python 3.10.12


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
x

[tensor([[[[ 0.9341, -0.9725,  0.9455,  ...,  0.9377,  0.9337,  0.9225],
           [ 0.8859, -0.9780,  0.9612,  ...,  0.9813,  0.9585,  0.9617],
           [ 1.1415, -0.9614,  1.2673,  ...,  1.2588,  1.2873,  1.2687],
           ...,
           [ 0.3158, -0.9757,  0.2982,  ...,  0.2956,  0.2999,  0.3034],
           [ 1.1350, -0.9894,  1.1331,  ...,  1.1316,  1.1272,  1.1398],
           [ 2.4030, -0.9733,  2.3993,  ...,  2.4090,  2.4070,  2.3928]],
 
          [[ 0.8983, -0.9902,  0.8961,  ...,  0.8879,  0.9018,  0.8858],
           [ 0.8302, -0.9688,  0.9245,  ...,  0.9165,  0.9294,  0.9309],
           [ 1.1136, -0.9618,  1.2183,  ...,  1.2334,  1.2090,  1.2340],
           ...,
           [ 0.3463, -0.9917,  0.3030,  ...,  0.2919,  0.3020,  0.3030],
           [ 1.1303, -0.9776,  1.1334,  ...,  1.1414,  1.1361,  1.1202],
           [ 2.4052, -0.9900,  2.4019,  ...,  2.4025,  2.4092,  2.4056]],
 
          [[ 0.8578, -0.9700,  0.8597,  ...,  0.8519,  0.8558,  0.8588],
           [ 